In [ ]:
#training for 20-50 ages
# Install Kaggle package
!pip install -q kaggle

# Upload your Kaggle API JSON file
from google.colab import files
files.upload()

# Create a directory for Kaggle and move your kaggle.json file there
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle (Replace <dataset-identifier> with the actual dataset identifier)
!kaggle datasets download -d mariafrenti/age-prediction

# Unzip the dataset (Replace <dataset-name>.zip with the actual zip file name)
!unzip age-prediction.zip

import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from glob import glob

# Paths to the dataset
train_dir = '/content/20-50/20-50/train'  # Replace with actual train directory path
test_dir = '/content/20-50/20-50/test'    # Replace with actual test directory path

# Count the number of images in the dataset
num_train_images = len(glob(train_dir + '/*/*.jpg'))
num_test_images = len(glob(test_dir + '/*/*.jpg'))
print(f"Number of training images: {num_train_images}")
print(f"Number of testing images: {num_test_images}")

# Data generators for training and testing
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)
# Choose between ResNet50 or EfficientNetB0
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# or
#base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1)(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()
# Model checkpoint to save best weights
checkpoint = ModelCheckpoint('best_model.weights.h5', monitor='val_mae', save_best_only=True, save_weights_only=True)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_mae', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=3,
    callbacks=[checkpoint, early_stopping]
)

# Load the best weights
model.load_weights('best_model.weights.h5')
# Evaluate the model on the test set
# test_loss, test_mae = model.evaluate(test_generator, verbose=1)
# print(f'Test Loss: {test_loss}')
# print(f'Test MAE: {test_mae}')
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Make predictions
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)  # If your predictions are probabilities, get the class with the highest probability

# Get true labels from the test set
y_true = test_generator.classes

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.show()
from google.colab import files
model.save('age_prediction_model_resnet2.keras')
# Download the model file
files.download('age_prediction_model_resnet2.keras')
model.save('age_prediction_model_resnet2.h5')
# Download the model file
files.download('age_prediction_model_resnet2.h5')

In [2]:
#training the combined model for 1-19
import tensorflow as tf
print(tf.__version__)
#for ages 1-19
# Install Kaggle package
#!pip install -q kaggle

# Upload your Kaggle API JSON file
from google.colab import files
files.upload()

# Create a directory for Kaggle and move your kaggle.json file there
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle (Replace <dataset-identifier> with the actual dataset identifier)
!kaggle datasets download -d felixjoseph7/Age-data-1-19

# Unzip the dataset (Replace <dataset-name>.zip with the actual zip file name)
!unzip Age-data-1-19.zip


2.17.0


In [ ]:
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.models import load_model

# Load your existing model trained on ages 20-50
model = load_model('age_prediction_model_resnet2.keras')


In [ ]:
# Paths to the new dataset (ages 1-19)
train_dir_1_19 = '/content/1-19/train'  # Replace with actual train directory path
test_dir_1_19 = '/content/1-19/test'    # Replace with actual test directory path

# Create data generators for the new dataset
train_datagen_1_19 = ImageDataGenerator(rescale=1./255, horizontal_flip=True, validation_split=0.2)
test_datagen_1_19 = ImageDataGenerator(rescale=1./255)

train_generator_1_19 = train_datagen_1_19.flow_from_directory(
    train_dir_1_19,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

validation_generator_1_19 = train_datagen_1_19.flow_from_directory(
    train_dir_1_19,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

test_generator_1_19 = test_datagen_1_19.flow_from_directory(
    test_dir_1_19,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)


In [ ]:
# Model checkpoint to save best weights during fine-tuning
checkpoint_1_19 = ModelCheckpoint('best_model_1_19.weights.keras', monitor='val_mae', save_best_only=True, save_weights_only=True)

# Fine-tune the model on the ages 1-19 dataset
history_1_19 = model.fit(
    train_generator_1_19,
    validation_data=validation_generator_1_19,
    epochs=5,  # Increase if needed
    callbacks=[checkpoint_1_19, EarlyStopping(monitor='val_mae', patience=3, restore_best_weights=True)]
)

# Load the best weights from this fine-tuning stage
model.load_weights('best_model_1_19.weights.keras')


In [ ]:
# Recreate the data generators for ages 20-50 to mix with the new dataset
train_generator_20_50 = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

validation_generator_20_50 = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

# Fine-tune the model on the combined dataset (ages 1-50)
history_combined = model.fit(
    train_generator_20_50,
    validation_data=validation_generator_20_50,
    epochs=5,  # Continue training for a few more epochs
    callbacks=[checkpoint_1_19, EarlyStopping(monitor='val_mae', patience=3, restore_best_weights=True)]
)


In [ ]:
# Save the updated model
model.save('age_prediction_model_combined50.keras')
files.download('age_prediction_model_combined50.keras')


In [ ]:
# Evaluate on the new test set (ages 1-19)
test_loss_1_19, test_mae_1_19 = model.evaluate(test_generator_1_19)
print(f'Test Loss (ages 1-19): {test_loss_1_19}')
print(f'Test MAE (ages 1-19): {test_mae_1_19}')

# Evaluate on the original test set (ages 20-50)
test_loss_20_50, test_mae_20_50 = model.evaluate(test_generator_20_50)
print(f'Test Loss (ages 20-50): {test_loss_20_50}')
print(f'Test MAE (ages 20-50): {test_mae_20_50}')
